In [ ]:
import sys
sys.path.append("../..")
import torch
DIFF_STEPS = 60
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
from train import Config, LitQuetzal

kwargs = {
    "bsz": 100,
    "device": DEVICE,
    "num_steps": DIFF_STEPS,
    "pbar": True,
    "max_len": 32,
}

ckpt = "../../checkpoints/original.ckpt"

lit = LitQuetzal.load_from_checkpoint(ckpt, map_location=DEVICE)
model = lit.ema.module
model.eval();

out = model.generate(**kwargs)

 78%|███████▊  | 25/32 [00:06<00:01,  3.67it/s]


In [12]:
from draw import show_grid
show_grid(out[0], 5, 5, offset=50)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [11]:
out[0][25]

Molecule(num_atoms=20, atoms=torch.Size([25]), coords=torch.Size([25, 3]))

In [ ]:
from draw import show_traj
# b_idx = 0+25
b_idx = 12+25
view = out[0][b_idx].show()
view = show_traj(out, b_idx, interval=20)
# view.rotate(30, {'x': 2.27, 'z': -4.63})
# view.rotate(180, {'x': 1})
view

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [31]:
from chem import Molecule

M_full = out[0][b_idx].clone()

opacity = 1.0
mol_opacity = 0.7

T = out[1].shape[2]


import py3Dmol

view = py3Dmol.view(width=400*7, height=400*3, viewergrid=(3, 7))
# view.rotate(30, {'x': 2.27, 'z': -4.63})
# view.rotate(180, {'x': 1})
view.zoom(0.75)
# view.translate(-5, -40)


from rdkit import Chem

for t in range(19):
    row = t // 7
    col = t % 7
    
    view.addModel(Molecule(M_full.atoms[:t], M_full.coords[:t]).xyzfile, "xyz", viewer=(row, col))
    style = {"stick": {"radius": .2, "opacity": mol_opacity}, "sphere": {"scale": .2, "opacity": mol_opacity}}
    view.setStyle({"model": -1}, style, viewer=(row, col))

    traj_atoms = out[0][b_idx].atoms[t].repeat(T)
    traj_coords = out[1][b_idx][t]
    view.addModel(Molecule(traj_atoms, traj_coords).xyzfile, "xyz", viewer=(row, col))
    style = {"sphere": {"scale": 0.2, "opacity": opacity}}
    view.setStyle({"model": -1}, style, viewer=(row, col))

from draw import make_html
make_html(view, "gen_sequence.html")
view

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [32]:
from chem import Molecule

M_full = out[0][b_idx].clone()

opacity = 1.0
mol_opacity = 0.7

T = out[1].shape[2]


import py3Dmol

view = py3Dmol.view(width=400*7, height=400*3, viewergrid=(3, 7))
# view.rotate(30, {'x': 2.27, 'z': -4.63})
# view.rotate(180, {'x': 1})
view.zoom(1.10)
# view.translate(-5, -60)


from rdkit import Chem

for t in range(19):
    row = t // 7
    col = t % 7

    a = M_full.atoms[:t]
    c = M_full.coords[:t]

    view.addModel(Molecule(M_full.atoms[:t], M_full.coords[:t]).xyzfile, "xyz", viewer=(row, col))
    style = {"stick": {"radius": .2, "opacity": mol_opacity}, "sphere": {"scale": .2, "opacity": mol_opacity}}
    view.setStyle({"model": -1}, style, viewer=(row, col))

    kwargs = {
        "bsz": 2000,
        "device": DEVICE,
        "num_steps": 60,
        "pbar": False,
        "prefix": (a, c),
        "max_len": t+2,
    }

    nextout = model.generate(**kwargs)

    newa = nextout[0].atoms[:, -1]
    newc = nextout[0].coords[:, -1]
    view = Molecule(a, c).show(view=view, viewer=(row, col), zoom=False)
    view.addModel(Molecule(newa, newc).xyzfile, "xyz", viewer=(row, col))
    style = {"cross": {"radius": 0.2}}
    view.setStyle({"model": -1}, style, viewer=(row, col))
    style = {"cross": {"radius": 0.2, "color": "black"}}
    view.setStyle({"model": -1, "atom": "H"}, style, viewer=(row, col))
    
from draw import make_html
make_html(view, "distribution_sequence.html")
view

3Dmol.js failed to load for some reason. Please check your browser console for error messages.